# Проверка базовых простейщих моделей на корпусе crowdsource_for_grant

In [1]:
import sys
import os
source_path = os.path.split(os.path.abspath(""))[0]
sys.path.append(source_path)

In [2]:
import numpy as np
import pandas as pd

from simpletransformers.classification import ClassificationModel, ClassificationArgs

from datasets import load_dataset

from src.data.evaluate import evaluate_all_tasks

## Данные

In [3]:
SMALL_DS = True

In [4]:
inp_tr = load_dataset("sagteam/author_profiling", split="train").to_pandas()
inp_vl = load_dataset("sagteam/author_profiling", split="validation").to_pandas()
inp_ts = load_dataset("sagteam/author_profiling", split="test").to_pandas()

if SMALL_DS:
    inp_tr = inp_tr.sample(frac=1)
    inp_vl = inp_vl.sample(frac=1)
    inp_tr = inp_tr.iloc[:1000]
    inp_vl = inp_vl.iloc[:1000]

inp_tr = inp_tr.replace("None", None)
inp_vl = inp_vl.replace("None", None)
inp_ts = inp_ts.replace("None", None)

No config specified, defaulting to: author_profiling/main
Reusing dataset author_profiling (/home/saguser/.cache/huggingface/datasets/sagteam___author_profiling/main/2.0.1/6c07c7d8f4349ec5178d8f0c93b529c36bd3a2c3cb31504a8e8756fd34bbf064)
No config specified, defaulting to: author_profiling/main
Reusing dataset author_profiling (/home/saguser/.cache/huggingface/datasets/sagteam___author_profiling/main/2.0.1/6c07c7d8f4349ec5178d8f0c93b529c36bd3a2c3cb31504a8e8756fd34bbf064)
No config specified, defaulting to: author_profiling/main
Reusing dataset author_profiling (/home/saguser/.cache/huggingface/datasets/sagteam___author_profiling/main/2.0.1/6c07c7d8f4349ec5178d8f0c93b529c36bd3a2c3cb31504a8e8756fd34bbf064)


In [5]:
x_tr = inp_tr["text"].values
x_vl = inp_vl["text"].values
x_ts = inp_ts["text"].values

In [6]:
l_columns = ["age_group", "gender", "no_imitation", "age_imitation",
             "gender_imitation", "style_imitation"]

In [7]:
# wandb_project = "deb-GraphHyperOpt"
wandb_project = None

## Модели

### RuBERT модель

In [8]:
# Model configuration for run with RuBERT model
# d_model_config = {"model_type": "transformer",
#                   "model": "bert",
#                   "model_name": "DeepPavlov/rubert-base-cased",
#                   "model_args":{
#                       "overwrite_output_dir": True,
#                       "output_dir": "g1_model",
#                       "save_best_model": True,
#                       "save_eval_checkpoints": False,
#                       "save_model_every_epoch": False,
#                       "save_optimizer_and_scheduler": False,
#                       "save_steps": -1,
#                       "n_gpu": 4,
#                       "train_batch_size": 8*8,
#                       "learning_rate": 0.00001*4,
#                       "evaluate_during_training": True,
#                       "evaluate_each_epoch": True,
#                       "num_train_epochs": 10,
#                       "max_seq_length": 512,
#                       "eval_batch_size": 8*4}}

# Model configuration for run with tiny-rubert
# d_model_config = {"model_type": "transformer",
#                   "model": "bert",
#                   "model_name": "cointegrated/rubert-tiny",
#                   "model_args":{
#                       "overwrite_output_dir": True,
#                       "output_dir": "g1_model",
#                       "save_best_model": True,
#                       "save_eval_checkpoints": False,
#                       "save_model_every_epoch": False,
#                       "save_optimizer_and_scheduler": False,
#                       "save_steps": -1,
#                       "n_gpu": 4,
#                       "train_batch_size": 8*8,
#                       "learning_rate": 0.00001*4,
#                       "evaluate_during_training": True,
#                       "evaluate_each_epoch": True,
#                       "num_train_epochs": 10,
#                       "max_seq_length": 512,
#                       "eval_batch_size": 8*4}}

# Debug model configuration
d_model_config = {"model_type": "transformer", 
                  "model": "bert",
                  "model_name": "cointegrated/rubert-tiny",
                  "model_args":{
                      "overwrite_output_dir": True,
                      "output_dir": "g1_model",
                      "save_best_model": True,
                      "save_eval_checkpoints": False,
                      "save_model_every_epoch": False,
                      "save_optimizer_and_scheduler": False,
                      "save_steps": -1,
                      # "n_gpu": 4,
                      "train_batch_size": 1,
                      "learning_rate": 0.00001*4,
                      "evaluate_during_training": True,
                      "evaluate_each_epoch": True,
                      "num_train_epochs": 3,
                      "max_seq_length": 32,
                      "eval_batch_size": 8*4}}

res_rubert = {"id": inp_ts["id"], "author_id": inp_ts["author_id"]}
for task_name in l_columns:                  
    df_task_tr = pd.DataFrame(inp_tr[inp_tr[task_name].notnull()][["text", task_name]], copy=True)
    df_task_vl = pd.DataFrame(inp_vl[inp_vl[task_name].notnull()][["text", task_name]], copy=True)
    
    df_task_tr = df_task_tr.rename(columns={task_name: "labels"})
    df_task_tr["labels"] = df_task_tr["labels"].astype(str)
    df_task_vl = df_task_vl.rename(columns={task_name: "labels"})
    df_task_vl["labels"] = df_task_vl["labels"].astype(str)
    
    labels_list = list(df_task_tr["labels"].unique())
    
    model_args = ClassificationArgs(labels_list=labels_list, **d_model_config["model_args"])
    model = ClassificationModel(d_model_config["model"], d_model_config["model_name"], args=model_args,
                                num_labels=len(labels_list), use_cuda=True)
    model.train_model(df_task_tr, eval_df=df_task_vl, verbose=True)
    model = ClassificationModel(d_model_config["model"], "outputs/best_model/", use_cuda=True, num_labels=len(labels_list))
    pred, _ = model.predict(x_ts.tolist())
    
    res_rubert[task_name] = pred

df_res_rubert = pd.DataFrame(res_rubert)
d_scores_rubert = evaluate_all_tasks(inp_ts, df_res_rubert, wandb_project=wandb_project, wandb_config=d_model_config)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Running Epoch 2 of 3:   0%|          | 0/407 [00:00<?, ?it/s]

  0%|          | 0/360 [00:00<?, ?it/s]

  0%|          | 0/2564 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/saguser/anaconda3/envs/graph_evalution_model/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [9]:
df_res_rubert.to_csv("df_res_rubert.csv")
inp_ts.to_csv("inp_ts.csv")

In [10]:
print("F1-score weighted, average: {0:0.2f}".format(d_scores_rubert["average"]["all"]["f1_weighted"]))
print("")
for k, v in d_scores_rubert["details"].items():
    print("Task: {0}".format(k))
    print(v["all"]["classification_report"])
    print("")

F1-score weighted, average: 0.54

Task: age_group
              precision    recall  f1-score   support

        0-19       0.00      0.00      0.00       195
       20-29       0.44      0.94      0.60      1131
       30-39       0.24      0.05      0.09       683
       40-49       0.00      0.00      0.00       351
         50+       0.00      0.00      0.00       204

    accuracy                           0.43      2564
   macro avg       0.14      0.20      0.14      2564
weighted avg       0.26      0.43      0.29      2564


Task: gender
              precision    recall  f1-score   support

      female       0.61      0.37      0.46      1290
        male       0.54      0.77      0.64      1274

    accuracy                           0.57      2564
   macro avg       0.58      0.57      0.55      2564
weighted avg       0.58      0.57      0.55      2564


Task: no_imitation
                    precision    recall  f1-score   support

  no_any_imitation       0.63      0.39

In [11]:
# import wandb
# wandb.finish()

In [12]:
print("Выполнено")

Выполнено
